# Managing Team State - Saving and Loading Teams



isme basically ham kaam krne ke baad state ko save kr dete hain, if agar reset kiya agent team ko agar hme fir se pehle state jisha hona hai to whi save state ko load krnege to agent ke paas purani wali memory or state aa jayegi

In [2]:
import asyncio
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage

model_client = OllamaChatCompletionClient(model="gemma2:latest")

In [3]:
from autogen_agentchat.conditions import MaxMessageTermination
agent1 = AssistantAgent(
    name = "writer1",
    model_client=model_client,
    system_message="You are a contemporary writer. Give the output in less than 30 words."
)

agent2 = AssistantAgent(
    name = "writer2",
    model_client=model_client,
    system_message="You are a classic writer. Give the output in less than 30 words."
)

termination_condition = MaxMessageTermination(max_messages=3)
agent_team = RoundRobinGroupChat(participants=[agent1,agent2], termination_condition=termination_condition)

In [4]:
stream = agent_team.run_stream(task="Write a poem about the sea in 3 lines")

In [5]:
from autogen_agentchat.ui import Console
await Console(stream)

---------- TextMessage (user) ----------
Write a poem about the sea in 3 lines
---------- TextMessage (writer1) ----------
Salt air whispers secrets untold,
Waves crash and sigh, an endless gold.
The ocean breathes, both fierce and bold. 



---------- TextMessage (writer2) ----------
The boundless blue, a restless soul,
Where silver foam on sunbeams stroll,
And crashing waves reach heaven's goal.  



TaskResult(messages=[TextMessage(id='de6b430d-9f92-4fd1-a47e-e824695e6417', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 34, 31, 601799, tzinfo=datetime.timezone.utc), content='Write a poem about the sea in 3 lines', type='TextMessage'), TextMessage(id='0b44c10a-a7e6-4288-b1e4-0d84296d5b86', source='writer1', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=29), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 34, 52, 861072, tzinfo=datetime.timezone.utc), content='Salt air whispers secrets untold,\nWaves crash and sigh, an endless gold.\nThe ocean breathes, both fierce and bold. \n\n\n', type='TextMessage'), TextMessage(id='a5b1b9e1-9790-41d8-a334-a8cf3252ded0', source='writer2', models_usage=RequestUsage(prompt_tokens=65, completion_tokens=30), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 4, 143390, tzinfo=datetime.timezone.utc), content="The boundless blue, a restless soul,\nWhere silver foam

In [6]:
team_state = await agent_team.save_state()
print("Team state saved.")
print(team_state)


Team state saved.
{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'writer1': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Salt air whispers secrets untold,\nWaves crash and sigh, an endless gold.\nThe ocean breathes, both fierce and bold. \n\n\n', 'thought': None, 'source': 'writer1', 'type': 'AssistantMessage'}]}}, 'message_buffer': [{'id': 'a5b1b9e1-9790-41d8-a334-a8cf3252ded0', 'source': 'writer2', 'models_usage': {'prompt_tokens': 65, 'completion_tokens': 30}, 'metadata': {}, 'created_at': '2025-10-10T10:35:04.143390Z', 'content': "The boundless blue, a restless soul,\nWhere silver foam on sunbeams stroll,\nAnd crashing waves reach heaven's goal.  \n", 'type': 'TextMessage'}]}, 'writer2': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {

In [7]:
await agent_team.reset()

In [8]:
stream = agent_team.run_stream(task="what is the last line of poem you wrote")

In [9]:
await Console(stream)


---------- TextMessage (user) ----------
what is the last line of poem you wrote


---------- TextMessage (writer1) ----------
The city sighs, a concrete lullaby under neon skies.  

---------- TextMessage (writer2) ----------
And dreams of cobblestones whisper beneath steel's cold sigh.  



TaskResult(messages=[TextMessage(id='e512acc4-375c-44ab-a48a-ec4baf8a783a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 4, 266336, tzinfo=datetime.timezone.utc), content='what is the last line of poem you wrote', type='TextMessage'), TextMessage(id='0053d592-c1b5-4323-95f2-cbe380ce6ba2', source='writer1', models_usage=RequestUsage(prompt_tokens=35, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 10, 682200, tzinfo=datetime.timezone.utc), content='The city sighs, a concrete lullaby under neon skies.  \n', type='TextMessage'), TextMessage(id='fe9b7d80-7992-46dd-a33b-bedbb0c0046a', source='writer2', models_usage=RequestUsage(prompt_tokens=50, completion_tokens=17), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 18, 331217, tzinfo=datetime.timezone.utc), content="And dreams of cobblestones whisper beneath steel's cold sigh.  \n", type='TextMessage')], stop_reason='Maximum number of

In [10]:
await agent_team.load_state(team_state)

In [11]:
stream = agent_team.run_stream(task="what is the last line of poem you wrote")
await Console(stream)

---------- TextMessage (user) ----------
what is the last line of poem you wrote
---------- TextMessage (writer1) ----------
The last line of the poem is: "And crashing waves reach heaven's goal." 

---------- TextMessage (writer2) ----------
You are absolutely right!  I am still under development and learning to pay better attention to detail.  Thank you for catching that. 😊 



TaskResult(messages=[TextMessage(id='ddea9579-e9d1-4e96-91dc-685fecdbe0f5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 18, 388512, tzinfo=datetime.timezone.utc), content='what is the last line of poem you wrote', type='TextMessage'), TextMessage(id='63ad1064-e7bf-4a1b-923f-9ee5b808c90a', source='writer1', models_usage=RequestUsage(prompt_tokens=112, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 35, 810468, tzinfo=datetime.timezone.utc), content='The last line of the poem is: "And crashing waves reach heaven\'s goal." \n', type='TextMessage'), TextMessage(id='7fc4f169-c1f6-4eb6-9d8c-485b08cccd63', source='writer2', models_usage=RequestUsage(prompt_tokens=134, completion_tokens=31), metadata={}, created_at=datetime.datetime(2025, 10, 10, 10, 35, 54, 103854, tzinfo=datetime.timezone.utc), content='You are absolutely right!  I am still under development and learning to pay better attention to de

In [12]:
import json

In [13]:
with open('team_state.json','w') as f:
    json.dump(team_state,f)

In [14]:
team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'writer1': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Salt air whispers secrets untold,\nWaves crash and sigh, an endless gold.\nThe ocean breathes, both fierce and bold. \n\n\n',
       'thought': None,
       'source': 'writer1',
       'type': 'AssistantMessage'}]}},
   'message_buffer': [{'id': 'a5b1b9e1-9790-41d8-a334-a8cf3252ded0',
     'source': 'writer2',
     'models_usage': {'prompt_tokens': 65, 'completion_tokens': 30},
     'metadata': {},
     'created_at': '2025-10-10T10:35:04.143390Z',
     'content': "The boundless blue, a restless soul,\nWhere silver foam on sunbeams stroll,\nAnd crashing waves reach heaven's goal.  \n",
     'type': 'TextMessage'}]},
  'writer2'